In [83]:
from pyspark.sql.types import FloatType, DoubleType
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import desc
import numpy as np
import pyspark.sql.functions as F
#import matplotlib.pyplot as plt
from pyspark import SparkContext
from nltk.stem import PorterStemmer

VBox()

In [84]:
spark = SparkSession.builder.appName("Assignment2").getOrCreate()

VBox()

Downloading the data

In [85]:
dic = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
raw_data_df = spark.read.csv(dic,header=True,sep='\t')

#raw_data_df.show(2, True)

VBox()

In [86]:
organized_data = raw_data_df.select(['product_id', 'review_body', 'star_rating', 'review_id'])
#organized_data.show()

VBox()

In [87]:
product_review_number = organized_data.groupBy('product_id').count()

VBox()

In [88]:
sort_by_reviews = product_review_number.sort(desc('count'))

VBox()

In [89]:
product_with_most_reviews = sort_by_reviews.limit(1)

VBox()

Selected top product is used further for processing

In [90]:
Top_product = product_with_most_reviews.join(organized_data,product_with_most_reviews.product_id == organized_data.product_id,how='right')

VBox()

In [92]:
Top_product_filter = Top_product.filter(product_with_most_reviews.product_id.isNotNull()).select([product_with_most_reviews.product_id, 'review_body', 'star_rating', 'review_id'])

VBox()

In [94]:
positive_reviews = Top_product_filter.filter("star_rating>=4").select(['product_id','review_body'])
negative_reviews = Top_product_filter.filter("star_rating<=2").select(['product_id','review_body'])

VBox()

In [95]:
import re

VBox()

In [96]:
def filterlines(line):
    temp_list = []
    review_body = re.split('[?.!]', str(line))
    for line in review_body:
        line = line.strip()
        if line != '':
            temp_list.append(line)
    return (temp_list)

VBox()

In [97]:
positive_reviews = positive_reviews.rdd.map(lambda x : x)
negative_reviews = negative_reviews.rdd.map(lambda x : x)

VBox()

In [98]:
positive_reviews_list = positive_reviews.map(lambda x : (x[0], filterlines(x[1])))
negative_reviews_list = negative_reviews.map(lambda x : (x[0], filterlines(x[1])))

VBox()

In [99]:
def flatlist(listitem):
    flat_list = [item for sublist in listitem for item in sublist]
    return flat_list

VBox()

In [100]:
positive_reviews_list = positive_reviews_list.groupByKey().map(lambda x : (x[0], list(x[1])))
negative_reviews_list = negative_reviews_list.groupByKey().map(lambda x : (x[0], list(x[1])))

VBox()

In [101]:
positive_reviews_list = positive_reviews_list.map(lambda x : (x[0], flatlist(x[1])))
negative_reviews_list = negative_reviews_list.map(lambda x : (x[0], flatlist(x[1])))

VBox()

final_positive_df and final_negative_df contains product id and a list of sentences taken from the review. we can use this list of sentences for futher analysis


In [105]:
final_positive_RB = positive_reviews_list.take(1)[0][1]
final_negative_RB = negative_reviews_list.take(1)[0][1]

VBox()

Removing Stop_words and lemmetizer is used to convert words into their base words

In [106]:
ps = PorterStemmer()

VBox()

In [107]:
def stopword(word):
    stpwrds = ["a","about","above","after","again","against","all","am","an","and","any","are","aren\'t","as","at","be","because","been","before","being","below","between","both","but","by","can\'t","cannot","could","couldn\'t","did","didn\'t","do","does","doesn\'t","doing","don\'t","down","during","each","few","for","from","further","had","hadn\'t","has","hasn\'t","have","haven\'t","having","he","he\'d","he\'ll","he\'s","her","here","here\'s","hers","herself","him","himself","his","how","how\'s","i","i\'d","i\'ll","i\'m","i\'ve","if","in","into","is","isn\'t","it","it\'s","its","itself","let\'s","me","more","most","mustn\'t","my","myself","no","nor","not","of","off","on","once","only","or","other","ought","our","ours","out","over","own","same","shan\'t","she","she\'d","she\'ll","she\'s","should","shouldn\'t","so","some","such","than","that","that\'s","the","their","theirs","them","themselves","then","there","there\'s","these","they","they\'d","they\'ll","they\'re","they\'ve","this","those","through","to","too","under","until","up","very","was","wasn\'t","we","we\'d","we\'ll","we\'re","we\'ve","were","weren\'t","what","what\'s","when","when\'s","where","where\'s","which","while","who","who\'s","whom","why","why\'s","with","won\'t","would","wouldn\'t","you","you\'d","you\'ll","you\'re","you\'ve","your","yours","yourself","yourselves","br"]
    if word.lower() not in stpwrds:
        word = re.sub('[^A-Za-z0-9]+', '', word)
        return ps.stem(word)
    else:
        return ''

VBox()

In [108]:
new_positive_lines=[]
new_negative_lines=[]
temp_rd=[]
temp_line = []
for line in final_positive_RB:
    new_positive_lines=line.split(' ')
    for words in new_positive_lines:
        words = stopword(words)
        words = ps.stem(words)
        words = re.sub('[^A-Za-z0-9]+', '', words)
        if words != '':
            temp_line.append(words)
    if temp_line != '':
        if len(temp_line)==1:
            if not re.match('^[0-9\.\ ]*$',temp_line[0]):
                temp_rd.append(temp_line)
        else:
            temp_rd.append(temp_line)
    temp_line = []
final_positive_RB=temp_rd

temp_rd=[]
temp_line = []

for line in final_negative_RB:
    new_negative_lines=line.split(' ')
    for words in new_negative_lines:
        words = stopword(words)
        words = ps.stem(words)
        words = re.sub('[^A-Za-z0-9]+', '', words)
        if words != '':
            temp_line.append(words)
    if temp_line != '':
        if len(temp_line)==1:
            if not re.match('^[0-9\.\ ]*$',temp_line[0]):
                temp_rd.append(temp_line)
        else:
            temp_rd.append(temp_line)
    temp_line = []
final_negative_RB=temp_rd


VBox()

In [109]:
#creating wordset for positive and negative words

positivewordset = {}
negativewordset = {} 

for line in final_positive_RB:
    positivewordset = set(positivewordset).union(set(line))
for line in final_negative_RB:
    negativewordset = set(negativewordset).union(set(line))

VBox()

Coverting word set to dict

In [110]:
positivewordset = dict.fromkeys(positivewordset, 0)

VBox()

In [111]:
negativewordset = dict.fromkeys(negativewordset, 0)

VBox()

Creating dictionaries to keep word count

In [112]:
newpositiveword = []

for line in final_positive_RB:
    temp = {}
    temp = set(temp).union(set(line))
    temp = dict.fromkeys(temp, 0)
    for word in line:
        temp[word]+=1
    newpositiveword.append(temp)

VBox()

In [113]:
newnegativeword = []

for line in final_negative_RB:
    temp = {}
    temp = set(temp).union(set(line))
    temp = dict.fromkeys(temp, 0)
    for word in line:
        temp[word]+=1
    newnegativeword.append(temp)

VBox()

Normalizing tf

In [116]:
for line in newpositiveword:
    linelen = len(line)
    for key, value in line.items():
        line[key] = round((value/linelen),3)

VBox()

In [117]:
for line in newnegativeword:
    linelen = len(line)
    for key, value in line.items():
        line[key] = round((value/linelen),3)

VBox()

Calculating idf

In [118]:
for line in final_positive_RB:
    for word in line:
        positivewordset[word]+=1

VBox()

In [119]:
import math

for key,value in positivewordset.items():
    positivewordset[key] = round(math.log10(len(newpositiveword)/value), 3)

VBox()

In [120]:
for line in final_negative_RB:
    for word in line:
        negativewordset[word]+=1

VBox()

In [121]:
for key,value in negativewordset.items():
    negativewordset[key] = round(math.log10(len(newnegativeword)/value), 3)

VBox()

Calculating tf-idf

In [122]:
for line in newpositiveword:
    for key, value in line.items():
        line[key] = round((value * positivewordset[key]), 4)

VBox()

In [123]:
for line in newnegativeword:
    for key, value in line.items():
        line[key] = round((value * negativewordset[key]), 4)

VBox()

Finding cosine similarity

In [124]:
def calCosineSim(listA, listB):
    vectA = []
    vectB = []
    tempunionset = []
    
    #checking if there is any key in common
    if set(listA.keys()) & set(listB.keys()) == set():
        return 1.0000
    else:
        for key in dict(list(listA.items()) + list(listB.items())):
            tempunionset.append(key)

        for word in tempunionset:
            if word not in listA:
                vectA.append(0)
            else:
                vectA.append(listA[word])

            if word not in listB:
                vectB.append(0)
            else:
                vectB.append(listB[word])


        #find dot product between 2 sentences
        dotprodval = 0
        i = 0
        while i < len(vectA):
            dotprodval = dotprodval + (vectA[i]*vectB[i])
            i += 1
        
        #finding ||query|| = sqrt(d1[0]^2 * d1[1]^2.....d1[n]^2)
        tempA = 0
        for num in vectA:
            tempA = tempA + (num*num)
        tempA = math.sqrt(tempA)
        tempB = 0
        for num in vectB:
            tempB = tempB + (num*num)
        tempB = math.sqrt(tempB)

        #calculating cosine similarity
        tempmul = (tempA * tempB)
        cosSim = dotprodval / tempmul
        #cosine distance is 1-cosineSimilarity
        cosdist = 1 - cosSim
        return round(float(cosdist), 3)

VBox()

In [125]:
import numpy as np

VBox()

Creating 2D array to compute cosine distance

In [126]:
cosDistPositiverdd = sc.parallelize(newpositiveword)

VBox()

In [127]:
cosDistNegativerdd = sc.parallelize(newnegativeword)

VBox()

Now taking the cartesian product to find relevance between 2 sentences

In [128]:
cartesianPositiveProd = cosDistPositiverdd.cartesian(cosDistPositiverdd).cache()

VBox()

In [129]:
cartesianNegativeProd = cosDistNegativerdd.cartesian(cosDistNegativerdd).cache()

VBox()

CalCosineSim function is used to calculate the cosine distance between 2 sentences and same sentences are filtered out

In [132]:
calsimcartprodPositive = cartesianPositiveProd.map(lambda x : (calCosineSim(x[0], x[1]))).filter(lambda x: x!=0)

VBox()

In [133]:
calsimcartprodNegative = cartesianNegativeProd.map(lambda x : (calCosineSim(x[0], x[1]))).filter(lambda x: x!=0)

VBox()

In [134]:
temppositivedf = spark.createDataFrame(calsimcartprodPositive, FloatType()).cache()
tempnegativedf = spark.createDataFrame(calsimcartprodNegative, FloatType()).cache()

VBox()

In [135]:
from pyspark.sql.functions import col, avg
calmeanpositivedf = temppositivedf.select(avg(col("value"))).cache()
calmeannegativedf = tempnegativedf.select(avg(col("value"))).cache()

VBox()

Calculating the average cosine distance

In [136]:
calmeanpositivedf.take(1)

VBox()

[Row(avg(value)=0.9862500807503638)]

In [137]:
calmeannegativedf.take(1)

VBox()

[Row(avg(value)=0.9873073211189147)]

In [ ]:
spark.stop()